In [63]:
cd '/kaggle/input/c4w3-files'

/kaggle/input/c4w3-files


In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import re
import pdb
import time
import json
import torch
import string
import textwrap
import traceback
import itertools
import numpy as np 
import torch.nn as nn
from termcolor import colored
from torch.utils.data import Dataset, DataLoader

import transformer_utils_PT
import utils_PT

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
wrapper = textwrap.TextWrapper(width=70)
np.random.seed(42)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session\

/kaggle/input/flan-t5/pytorch/small/2/config.json
/kaggle/input/flan-t5/pytorch/small/2/spiece.model
/kaggle/input/flan-t5/pytorch/small/2/README.md
/kaggle/input/flan-t5/pytorch/small/2/tokenizer.json
/kaggle/input/flan-t5/pytorch/small/2/tf_model.h5
/kaggle/input/flan-t5/pytorch/small/2/tokenizer_config.json
/kaggle/input/flan-t5/pytorch/small/2/pytorch_model.bin
/kaggle/input/flan-t5/pytorch/small/2/special_tokens_map.json
/kaggle/input/flan-t5/pytorch/small/2/.gitattributes
/kaggle/input/flan-t5/pytorch/small/2/flax_model.msgpack
/kaggle/input/flan-t5/pytorch/small/2/generation_config.json
/kaggle/input/c4w3-files/c4-en-10k.json
/kaggle/input/c4w3-files/sentencepiece.model
/kaggle/input/c4w3-files/train-v2.0.json
/kaggle/input/c4w3-files/transformer_utils_PT.py
/kaggle/input/c4w3-files/inputs_targets_pairs_file.txt
/kaggle/input/c4w3-files/utils_PT.py
/kaggle/input/c4w3-files/c4-en-10k.jsonl
/kaggle/input/c4w3-files/data.txt


# Prepare the data for pretraining T5

## C4 dataset

In [65]:
# Load example jsons
with open('c4-en-10k.jsonl', 'r') as file:
    example_jsons = [json.loads(line.strip()) for line in file]
    
# Printing the examples to see how the data looks like
for i in range(5):
    print(f"example number {i+1}: \n\n{example_jsons[i]}\n")

example number 1: 

{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.'}

example number 2: 

{'text': 'Discussion in \'Mac OS X Lion (10.7)\' started by axboi87, Jan 20, 2012.\nI\'ve got a 500gb internal drive and a 240gb SSD.\nWhen trying to restore using disk utility i\'m given the erro

## Process C4


In [66]:
# Grab text field from dictionary
natural_language_texts = [example_json['text'] for example_json in example_jsons]

# Print the first text example
print(natural_language_texts[0])

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.


## Decode to Natural Language

In [67]:
# Uploading the sentencepiece tokenizer
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('/kaggle/input/c4w3-files/sentencepiece.model', clean_up_tokenization_spaces=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [68]:
# ID of the EOS token
eos = tokenizer.convert_tokens_to_ids("</s>")

print("EOS: " + str(eos))

EOS: 1


In [69]:
# Printing the encoding of each word to see how subwords are tokenized
tokenized_text = [(list(tokenizer.encode(word, add_special_tokens=False)), word) for word in natural_language_texts[2].split()]

print("Word\t\t-->\tTokenization")
print("-"*40)
for element in tokenized_text:
    print(f"{element[1]:<8}\t-->\t{element[0]}")

Word		-->	Tokenization
----------------------------------------
Foil    	-->	[4452, 173]
plaid   	-->	[30772]
lycra   	-->	[3, 120, 2935]
and     	-->	[11]
spandex 	-->	[8438, 26, 994]
shortall	-->	[710, 1748]
with    	-->	[28]
metallic	-->	[18813]
slinky  	-->	[3, 7, 4907, 63]
insets. 	-->	[16, 2244, 7, 5]
Attached	-->	[28416, 15, 26]
metallic	-->	[18813]
elastic 	-->	[15855]
belt    	-->	[6782]
with    	-->	[28]
O-ring. 	-->	[411, 18, 1007, 5]
Headband	-->	[3642, 3348]
included.	-->	[1285, 5]
Great   	-->	[1651]
hip     	-->	[5436]
hop     	-->	[13652]
or      	-->	[42]
jazz    	-->	[9948]
dance   	-->	[2595]
costume.	-->	[11594, 5]
Made    	-->	[6465]
in      	-->	[16]
the     	-->	[8]
USA.    	-->	[2312, 5]


In [70]:
# We can see that detokenize successfully undoes the tokenization
print(f"tokenized: {tokenizer.encode('Beginners')}\ndetokenized: {tokenizer.decode(tokenizer.encode('Beginners'), skip_special_tokens=True)}")

tokenized: [12847, 277, 1]
detokenized: Beginners


In [71]:
def get_sentinels(tokenizer, display=False):
    sentinels = {}
    vocab_size = tokenizer.vocab_size
    for i, char in enumerate(reversed(string.ascii_letters), 1):
        decoded_text = tokenizer.decode([vocab_size - i])
        
        # Sentinels, ex: <Z> - <a>
        sentinels[decoded_text] = f"<{char}>"
        
        if display:
            print(f"The sentinel is <{char}> and the decoded token is: {decoded_text}")
            
    return sentinels


def regex_replace(input_str, pattern, replace_with):
    return re.sub(pattern, replace_with, input_str)


def pretty_decode(encoded_str_list, sentinels, tokenizer):
    # If already a string, just do the replacements.
    if isinstance(encoded_str_list, str):
        for token, char in sentinels.items():
            encoded_str_list = regex_replace(encoded_str_list, token, char)
            
        return encoded_str_list
    
    return pretty_decode(tokenizer.decode(encoded_str_list, skip_special_tokens=True), sentinels, tokenizer)

In [72]:
sentinels = get_sentinels(tokenizer, display=True)

The sentinel is <Z> and the decoded token is: Internațional
The sentinel is <Y> and the decoded token is: erwachsene
The sentinel is <X> and the decoded token is: Cushion
The sentinel is <W> and the decoded token is: imunitar
The sentinel is <V> and the decoded token is: Intellectual
The sentinel is <U> and the decoded token is: traditi
The sentinel is <T> and the decoded token is: disguise
The sentinel is <S> and the decoded token is: exerce
The sentinel is <R> and the decoded token is: nourishe
The sentinel is <Q> and the decoded token is: predominant
The sentinel is <P> and the decoded token is: amitié
The sentinel is <O> and the decoded token is: erkennt
The sentinel is <N> and the decoded token is: dimension
The sentinel is <M> and the decoded token is: inférieur
The sentinel is <L> and the decoded token is: refugi
The sentinel is <K> and the decoded token is: cheddar
The sentinel is <J> and the decoded token is: unterlieg
The sentinel is <I> and the decoded token is: garanteaz
Th

In [73]:
pretty_decode('I want to dress up as an Intellectual this halloween.', sentinels, tokenizer)

'I want to dress up as an <V> this <b>.'

## Tokenizing and masking

In [74]:
def tokenize_and_mask(text, noise=0.15, randomizer=np.random.uniform, tokenizer=None):
    # Current sentinel number (starts at 0)
    cur_sentinel_num = 0
    
    # Inputs and targets
    inps, targs = [], []

    # Vocab_size
    vocab_size = tokenizer.vocab_size
    
    # EOS token id 
    # Must be at the end of each target!
    eos = tokenizer.convert_tokens_to_ids('</s>')
    
    # prev_no_mask is True if the previous token was NOT masked, False otherwise
    # set prev_no_mask to True
    prev_no_mask = True
    
    # Loop over the tokenized text
    for token in tokenizer.encode(text, add_special_tokens=False):
        
        # Generate a random value between 0 and 1
        rnd_val = randomizer()
        
        # Check if the noise is greater than a random value (weighted coin flip)
        if noise > rnd_val:
            
            # Check if previous token was NOT masked
            if prev_no_mask:
                
                # Current sentinel increases by 1
                cur_sentinel_num += 1
                
                # Compute end_id by subtracting current sentinel value out of the total vocabulary size
                end_id = vocab_size - cur_sentinel_num
                
                # Append end_id at the end of the targets
                targs.append(end_id)
                
                # Append end_id at the end of the inputs
                inps.append(end_id)
            
            # Append token at the end of the targets
            targs.append(token)
            
            # set prev_no_mask accordingly
            prev_no_mask = False
            
        else:
            # Append token at the end of the inputs
            inps.append(token)
            
            # Set prev_no_mask accordingly
            prev_no_mask = True
            
    targs.append(eos)
    return inps, targs

In [75]:
# Some logic to mock a np.random value generator
# Needs to be in the same cell for it to always generate same outpu
def testing_rnd():
    def dummy_generator():
        vals = np.linspace(0, 1, 10)
        cyclic_vals = itertools.cycle(vals)
        
        for _ in range(100):
            yield next(cyclic_vals)
            
    dumr = itertools.cycle(dummy_generator())
    
    def dummy_randomizer():
        return next(dumr)
    
    return dummy_randomizer


input_str = 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers.'
inps, targs = tokenize_and_mask(input_str, randomizer=testing_rnd(), tokenizer=tokenizer)
print(f"tokenized inputs - shape={len(inps)}:\n\n{inps}\n\ntargets - shape={len(targs)}:\n\n{targs}")

tokenized inputs - shape=53:

[31999, 15068, 4501, 3, 12297, 3399, 16, 5964, 7115, 31998, 531, 25, 241, 12, 129, 394, 44, 492, 31997, 58, 148, 56, 43, 8, 1004, 6, 474, 31996, 39, 4793, 230, 5, 2721, 6, 1600, 1630, 31995, 1150, 4501, 15068, 16127, 6, 9137, 2659, 5595, 31994, 782, 3624, 14627, 15, 12612, 277, 5]

targets - shape=19:

[31999, 12847, 277, 31998, 9, 55, 31997, 3326, 15068, 31996, 48, 30, 31995, 727, 1715, 31994, 45, 301, 1]


In [76]:
print('Inputs: \n\n', pretty_decode(inps, sentinels, tokenizer))
print('\nTargets: \n\n', pretty_decode(targs, sentinels, tokenizer))

Inputs: 

 <Z> BBQ Class Taking Place in Missoul <Y> Do you want to get better at making <X>? You will have the opportunity, put <W> your calendar now. Thursday, September 22 <V> World Class BBQ Champion, Tony Balay <U>onestar Smoke Rangers.

Targets: 

 <Z> Beginners <Y>a! <X> delicious BBQ <W> this on <V>nd join <U> from L


## Creating the pairs 

In [77]:
# Apply tokenize_and_mask
inputs_targets_pairs = [tokenize_and_mask(text.encode('utf-8', errors='ignore').decode('utf-8'), tokenizer=tokenizer) 
                        for text in natural_language_texts[:2000]]

In [78]:
def display_input_target_pairs(inputs_targets_pairs, sentinels, 
                               wrapper=textwrap.TextWrapper(width=70), tokenizer=tokenizer):
    for i, inp_tar_pair in enumerate(inputs_targets_pairs, 1):
        inps, tgts = inp_tar_pair
        inps = pretty_decode(inps, sentinels, tokenizer)
        tgts = pretty_decode(tgts, sentinels, tokenizer)
        
        print(f"[{i}]\n\n"
              f"inputs:\n{wrapper.fill(inps)}\n\n"
              f"targets:\n{wrapper.fill(tgts)}\n\n\n")

# Print 3 samples. We print inputs with less than 100 tokens. It is just to give you and idea of the process        
display_input_target_pairs(filter(lambda x: len(x[0]) < 100, inputs_targets_pairs[:12]), sentinels)

[1]

inputs:
<Z>il plaid <Y>lycra <X> spandex shortall with metallic slinky
<W>sets. Attache <V> metallic elastic belt with O <U>ring. Head <T>
included. Great hip hop<S> jazz dance costume.<R> in the USA.

targets:
<Z> Fo <Y>  <X> and <W> in <V>d <U>- <T>band<S> or<R> Made



[2]

inputs:
I thought I was going to <Z> 3rd season <Y> Wire tonight. <X> there
was a commentary <W> 11, so I had to re <V>watch <U> Ground with <T>
commentary. Hopefully<S> can finish<R> season <Q>.

targets:
<Z> finish the <Y> of the <X> But <W> on episode <V>- <U> Middle <T>
the<S> I<R> the <Q> next weekend



[3]

inputs:
Pencarian <Z>FILM Untuk " <Y>eace <X>er 2017 <W> yuk mampir ke channel
say <V>. Edges <U> provides the l.. A corrupt cop makes one w.. <T>er
2017 <S>.. Náo Lon - Peace Break.. Please subscribe and hit..<R> in HD
at http://.. <Q> cannot believe I manage..

targets:
<Z>  <Y>P <X> Break <W>" <V>. <U> East <T> Peace Break<S> <R> uploaded
<Q> I





# Pretrain a T5 model using C4

##  Instantiate a new transformer model

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [80]:
# Define the model parameters
num_layers = 2
embedding_dim = 128
fully_connected_dim = 128
num_heads = 2
positional_encoding_length = 256

encoder_vocab_size = tokenizer.vocab_size
decoder_vocab_size = encoder_vocab_size

# Initialize the model
transformer = transformer_utils_PT.Transformer(
    num_layers, 
    embedding_dim, 
    num_heads, 
    fully_connected_dim,
    encoder_vocab_size, 
    decoder_vocab_size, 
    positional_encoding_length, 
    positional_encoding_length,
).to(device)

In [81]:
#learning_rate = transformer_utils_PT.CustomSchedule(embedding_dim)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

criterion = nn.CrossEntropyLoss(reduction='none')

## C4 pretraining

In [82]:
# Limit the size of the input and output data so this can run in this environment
encoder_maxlen = 150
decoder_maxlen = 50

inputs = tokenizer.pad([{"input_ids": x[0]} for x in inputs_targets_pairs], max_length=encoder_maxlen, padding='max_length')
inputs = torch.tensor([seq[:encoder_maxlen] for seq in inputs['input_ids']])
targets = tokenizer.pad([{"input_ids": x[1]} for x in inputs_targets_pairs], max_length=decoder_maxlen, padding='max_length')
targets = torch.tensor([seq[:decoder_maxlen] for seq in targets['input_ids']])

### Dataset

In [83]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs, self.targets = inputs, targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return (self.inputs[idx], self.targets[idx])
    

BATCH_SIZE = 64

train_dataset = CustomDataset(inputs, targets)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [84]:
epochs = 10

num_of_batches = len(train_loader)
epoch_losses = []

for epoch in range(epochs):
    running_loss = 0.0
    transformer.train()
    
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        tar_inp = targets[:, :-1].to(device)
        tar_real = targets[:, 1:].to(device)
        
        # Masks
        enc_padding_mask = transformer_utils_PT.create_padding_mask(inputs).to(device)
        look_ahead_mask = transformer_utils_PT.create_look_ahead_mask(tar_inp.shape[1], len(inputs) * num_heads).to(device)
        dec_padding_mask = transformer_utils_PT.create_padding_mask(inputs).to(device)
        
        # Your common forw/back prop loop
        optimizer.zero_grad()
        predictions, _ = transformer(
            inputs, tar_inp, True, enc_padding_mask,
            look_ahead_mask, dec_padding_mask
        )
        
        loss = transformer_utils_PT.masked_loss(predictions.to(device), tar_real)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    # Calculating epoch losses    
    epoch_losses.append(running_loss / num_of_batches)
        
    print(f'Epoch {epoch+1}, Loss {epoch_losses[-1]}')

Epoch 1, Loss 10.492356657981873
Epoch 2, Loss 10.357855945825577
Epoch 3, Loss 10.228851228952408
Epoch 4, Loss 10.104453593492508
Epoch 5, Loss 9.976301193237305
Epoch 6, Loss 9.853137820959091
Epoch 7, Loss 9.721908271312714
Epoch 8, Loss 9.594953566789627
Epoch 9, Loss 9.468185484409332
Epoch 10, Loss 9.342415690422058


### Uploading a pretrained model

In [85]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("/kaggle/input/flan-t5/pytorch/small/2", clean_up_tokenization_spaces=True)
model = T5ForConditionalGeneration.from_pretrained("/kaggle/input/flan-t5/pytorch/small/2", device_map="auto")

# Fine tune the T5 model for Question Answering

In [86]:
with open('train-v2.0.json', 'r') as f:
    example_jsons = json.load(f)
    

example_jsons = example_jsons['data']

print('Number of articles: ' + str(len(example_jsons)))

Number of articles: 442


In [87]:
example_article = example_jsons[0]

print("Title: " + example_article['title'])
print(example_article['paragraphs'][0])

Title: Beyoncé
{'qas': [{'question': 'When did Beyonce start becoming popular?', 'id': '56be85543aeaaa14008c9063', 'answers': [{'text': 'in the late 1990s', 'answer_start': 269}], 'is_impossible': False}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'id': '56be85543aeaaa14008c9065', 'answers': [{'text': 'singing and dancing', 'answer_start': 207}], 'is_impossible': False}, {'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'id': '56be85543aeaaa14008c9066', 'answers': [{'text': '2003', 'answer_start': 526}], 'is_impossible': False}, {'question': 'In what city and state did Beyonce  grow up? ', 'id': '56bf6b0f3aeaaa14008c9601', 'answers': [{'text': 'Houston, Texas', 'answer_start': 166}], 'is_impossible': False}, {'question': 'In which decade did Beyonce become famous?', 'id': '56bf6b0f3aeaaa14008c9602', 'answers': [{'text': 'late 1990s', 'answer_start': 276}], 'is_impossible': False}, {'question': 'In what R&B group was she the

## Creating a list of paired question and answers
## Parse the SQuaD 2.0 Dataset

In [88]:
def parse_squad(dataset):
    
    inputs, targets = [], []
    
    # Loop over all the articles
    for article in dataset:
        
        # Loop over each paragraph of each article
        for paragraph in article['paragraphs']:
            
            # Extract context from the paragraph
            context = paragraph['context']
            
            # Loop over each question of the given paragraph
            for qa in paragraph['qas']:
                
                # If this question is not impossible and there is at least one answer
                if(len(qa['answers']) > 0 and not (qa['is_impossible'])):
                    
                    # Create the question/context sequence
                    question_context = 'question: ' + qa['question'] + ' context: ' + context
                    
                    # Create the answer sequence. Use the text field of the first answer
                    answer = 'answer: ' + qa['answers'][0]['text']
                    
                    # Add the question_context to the inputs list
                    inputs.append(question_context)
                    
                    # Add the answer to the targets list
                    targets.append(answer)
                    
    return inputs, targets

In [89]:
inputs, targets = parse_squad(example_jsons)
print("Number of question/answer pairs: " + str(len(inputs)))

print('\nFirst Q/A pair:\n\ninputs: ' + colored(inputs[0], 'blue'))
print('\ntargets: ' + colored(targets[0], 'green'))
print('\nLast Q/A pair:\n\ninputs: ' + colored(inputs[-1], 'blue'))
print('\ntargets: ' + colored(targets[-1], 'green'))

Number of question/answer pairs: 86821

First Q/A pair:

inputs: question: When did Beyonce start becoming popular? context: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".

targets: answer: in the late 1990s

Last Q/A pair:

inputs: question: What is KMC an initialism of? context: Kathmandu Metropolitan City (KMC), in order to promote international r

In [90]:
# 40K pairs for training
inputs_train = inputs[:40000]
targets_train = targets[:40000]

# 5K pairs for testing
inputs_test = inputs[40000:]
targets_test = targets[40000:]

In [91]:
# Limit the size of the input and output data so this can run in this environment
encoder_maxlen = 150
decoder_maxlen = 50

inputs = tokenizer(inputs_train, max_length=encoder_maxlen, truncation=True, padding='max_length', return_tensors='pt', add_special_tokens=False).input_ids
targets = tokenizer(targets_train, max_length=decoder_maxlen, truncation=True, padding='max_length', return_tensors='pt').input_ids

BATCH_SIZE = 64

train_dataset = CustomDataset(inputs, targets)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Fine tune the T5 model

In [92]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [93]:
"""epochs = 10

num_of_batches = len(train_loader)
epoch_losses = []

for epoch in range(epochs):
    running_loss = 0.0
    model.train()
    
    for i, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        tar_inp = targets[:, :-1].to(device)
        tar_real = targets[:, 1:].to(device)
        
        #Masks
        enc_padding_mask = transformer_utils_PT.create_padding_mask(inputs).to(device)
        look_ahead_mask = transformer_utils_PT.create_look_ahead_mask(tar_inp.shape[1], len(inputs) * num_heads).to(device)
        dec_padding_mask = transformer_utils_PT.create_padding_mask(tar_inp).to(device=device, dtype=torch.bool)
        
        optimizer.zero_grad()
        predictions = model(
            input_ids=inputs, attention_mask=enc_padding_mask, decoder_input_ids=tar_inp,
            decoder_attention_mask=dec_padding_mask, return_dict=False
        )
        #pdb.set_trace()
        predictions = torch.transpose(predictions[0], 1, 2)
        loss = transformer_utils_PT.masked_loss(predictions.to(device), tar_real)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    epoch_losses.append(running_loss / num_of_batches)
        
    print(f'Epoch {epoch+1}, Loss {epoch_losses[-1]}')"""

"epochs = 10\n\nnum_of_batches = len(train_loader)\nepoch_losses = []\n\nfor epoch in range(epochs):\n    running_loss = 0.0\n    model.train()\n    \n    for i, (inputs, targets) in enumerate(train_loader):\n        inputs = inputs.to(device)\n        tar_inp = targets[:, :-1].to(device)\n        tar_real = targets[:, 1:].to(device)\n        \n        #Masks\n        enc_padding_mask = transformer_utils_PT.create_padding_mask(inputs).to(device)\n        look_ahead_mask = transformer_utils_PT.create_look_ahead_mask(tar_inp.shape[1], len(inputs) * num_heads).to(device)\n        dec_padding_mask = transformer_utils_PT.create_padding_mask(tar_inp).to(device=device, dtype=torch.bool)\n        \n        optimizer.zero_grad()\n        predictions = model(\n            input_ids=inputs, attention_mask=enc_padding_mask, decoder_input_ids=tar_inp,\n            decoder_attention_mask=dec_padding_mask, return_dict=False\n        )\n        #pdb.set_trace()\n        predictions = torch.transpose(p

### Loading finetuned pretrained flan-T5-small

In [94]:
#model.save_pretrained('/kaggle/working/flan-T5_finetuned', from_pt=True)
model = T5ForConditionalGeneration.from_pretrained('/kaggle/working/flan-T5_small_finetuned').to(device)

## Implement your own Question Answering model

In [96]:
# Define an example question
example_question = "question: What color is the sky? context: Sky is blue"

# Question is tokenized and padded
# Note that this is hardcoded here but you must implement this in the upcoming exercise
tokenized_padded_question = torch.tensor([[822, 10, 363, 945, 19, 8, 5796, 58, 2625, 10, 5643, 19, 1692, 0, 0]]).to(device)

# All answers begin with the string "answer: "
# Feel free to check that this is indeed the tokenized version of that string
tokenized_answer = torch.tensor([[1525, 10]]).to(device)

# Predict the next word using the transformer_utils.next_word function
# Notice that it expects the question, answer and model (in that order)
next_word = transformer_utils_PT.next_word(model, tokenized_padded_question, tokenized_answer)
print(f"Predicted next word is : {tokenizer.batch_decode(next_word)}")

# Concatenate predicted word with answer so far
answer_so_far = torch.cat([tokenized_answer, next_word], dim=-1)

print(f"Answer so far: {tokenizer.batch_decode(answer_so_far)}")

Predicted next word is : ['blue']
Answer so far: ['answer: blue']


# Implement the question answering function

In [44]:
def answer_question(question, model, tokenizer, encoder_maxlen=150, decoder_maxlen=50):
    # Tokenize, pad, truncate the question
    tokenized_question = (tokenizer(question, max_length=encoder_maxlen, truncation=True, padding='max_length', return_tensors='pt', add_special_tokens=False).input_ids).to(device)
    
    # Tokenize, pad, truncate the answer
    tokenized_answer = (tokenizer('answer: ', max_length=decoder_maxlen, truncation=True, padding='max_length', return_tensors='pt').input_ids).to(device)
    eos = tokenizer.convert_tokens_to_ids("</s>") 
    
    # Generate the answers until the max seq length is reached or the EOS token
    for i in range(decoder_maxlen):
        next_word = transformer_utils_PT.next_word(model, tokenized_question, tokenized_answer).to(device)
        
        tokenized_answer = torch.cat([tokenized_answer, next_word], axis=1)
        
        if next_word == eos:
            break

    return tokenized_answer

In [45]:
idx = 10408

result = answer_question(inputs_train[idx], model, tokenizer)
result = torch.squeeze(result)
print(colored(pretty_decode(result, sentinels, tokenizer), 'blue'))
print()
print(inputs_train[idx])
print(colored(targets_train[idx], 'green'))

answer: January 9, 1957

question: When was the Chechen-Ingush Autonomous Soviet Socialist Republic transferred from the Georgian SSR? context: On January 9, 1957, Karachay Autonomous Oblast and Chechen-Ingush Autonomous Soviet Socialist Republic were restored by Khrushchev and they were transferred from the Georgian SSR back to the Russian SFSR.
answer: January 9, 1957


In [47]:
idx = 311
result = answer_question(inputs_test[idx], model, tokenizer)
result = torch.squeeze(result)
print(colored(pretty_decode(result, sentinels, tokenizer), 'blue'))
print()
print(inputs_test[idx])
print(colored(targets_test[idx], 'green'))

answer: 15

question:  On what date was a state funeral held for Napoleon? context: In 1840, Louis Philippe I obtained permission from the British to return Napoleon's remains to France. On 15 December 1840, a state funeral was held. The hearse proceeded from the Arc de Triomphe down the Champs-Élysées, across the Place de la Concorde to the Esplanade des Invalides and then to the cupola in St Jérôme's Chapel, where it remained until the tomb designed by Louis Visconti was completed. In 1861, Napoleon's remains were entombed in a porphyry sarcophagus in the crypt under the dome at Les Invalides.
answer: 15 December 1840


In [48]:
idx = 1231
result = answer_question(inputs_test[idx], model, tokenizer)
result = torch.squeeze(result)
print(colored(pretty_decode(result, sentinels, tokenizer), 'blue'))
print()
print(inputs_test[idx])
print(colored(targets_test[idx], 'green'))

answer: 2001

question: When did the WIPO adopt 'The Geneva Declaration on the Future of the World Intellectual Property Organization'? context: The World Intellectual Property Organization (WIPO) recognizes that conflicts may exist between the respect for and implementation of current intellectual property systems and other human rights. In 2001 the UN Committee on Economic, Social and Cultural Rights issued a document called "Human rights and intellectual property" that argued that intellectual property tends to be governed by economic goals when it should be viewed primarily as a social product; in order to serve human well-being, intellectual property systems must respect and conform to human rights laws. According to the Committee, when systems fail to do so they risk infringing upon the human right to food and health, and to cultural participation and scientific benefits. In 2004 the General Assembly of WIPO adopted The Geneva Declaration on the Future of the World Intellectual P

In [51]:
idx = 32
result = answer_question(inputs_test[idx], model, tokenizer)
result = torch.squeeze(result)
print(colored(pretty_decode(result, sentinels, tokenizer), 'blue'))
print()
print(inputs_test[idx])
print(colored(targets_test[idx], 'green'))

answer: Russia

question: What nation did Napoleon invade in 1812 to enforce his blockade? context: Tensions over rising Polish nationalism and the economic effects of the Continental System led to renewed confrontation with Russia. To enforce his blockade, Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic collapse of the Grand Army, forcing the French to retreat, as well as leading to the widespread destruction of Russian lands and cities. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France. A chaotic military campaign in Central Europe eventually culminated in a large Allied army defeating Napoleon at the Battle of Leipzig in October. The next year, the Allies invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba. The Bourbons were restored to power and the French lost most of the territories that they had conquered since the R